In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [1]:
import numpy as np
import scipy.io as sio

In [2]:
data_path = "/workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat"
# data_path_tpfm = "/workspaces/deeprte/data/train/test_sin_1019.npz"

In [3]:
data = sio.loadmat(data_path)

In [4]:
data.keys()
for k,v in data.items():
    if isinstance(v, np.ndarray):
        print(k , v.shape)

ct (24, 1)
omega_prime (164, 12)
phi (10, 41, 41)
psi_bc (10, 164, 12)
psi_label (10, 41, 41, 24)
rv_prime (164, 12, 4)
scattering_kernel (10, 24, 24)
sigma_a (10, 41, 41)
sigma_t (10, 41, 41)
st (24, 1)
w_angle (24, 1)
x (1, 41)
y (1, 41)


In [5]:
from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline()
data = data_pipeline.process(data_path)

In [6]:
for k,v in data.items():
    if isinstance(v, np.ndarray):
        print(k , v.shape)

sigma (10, 41, 41, 2)
psi_label (10, 41, 41, 24)
phi (10, 41, 41)
boundary (10, 164, 12)
position_coords (41, 41, 2)
velocity_coords (24, 2)
phase_coords (41, 41, 24, 4)
scattering_kernel (10, 41, 41, 24, 24)
boundary_coords (164, 12, 4)
boundary_weights (164, 12)
velocity_weights (24,)


# batch
- sigma [10, 41*41, 2]
- boundary [10, 164*12]
- scattering_kernel [10, 41\*41\*24, 24]

# collocation
- phase_coords [41\*41\*24, 4]
- scattering_kernel [10, 41\*41\*24, 24]

# grid
- position_coords [41\*41, 2]
- velocity_coords [24, 2]
- boundary_coords [164\*12, 4]
- boundary_weights [164\*12]
- velocity_weights [24]

In [7]:
from deeprte.model.tf.rte_features import FEATURES, shape, _COLLOCATION_FEATURE_NAMES

2022-11-23 21:45:33.543959: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 21:45:33.714867: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
FEATURES

{'sigma': (tf.float32,
  ['num batch placeholder', 'num position coords placeholder', 2]),
 'boundary': (tf.float32,
  ['num batch placeholder', 'num boundary placeholder']),
 'position_coords': (tf.float32, ['num position coords placeholder', 2]),
 'velocity_coords': (tf.float32, ['num velocity placeholder', 2]),
 'phase_coords': (tf.float32, ['num phase placeholder', 4]),
 'scattering_kernel': (tf.float32,
  ['num batch placeholder',
   'num phase placeholder',
   'num velocity placeholder']),
 'boundary_coords': (tf.float32, ['num boundary placeholder', 4]),
 'boundary_weights': (tf.float32, ['num boundary placeholder']),
 'velocity_weights': (tf.float32, ['num velocity placeholder']),
 'psi_label': (tf.float32, ['num batch placeholder', 'num phase placeholder'])}

In [9]:
_COLLOCATION_FEATURE_NAMES

['phase_coords', 'scattering_kernel', 'psi_label']

In [10]:
from deeprte.model.tf.feature_transform import sample_phase_points
from deeprte.model.tf.rte_dataset import np_to_tensor_dict,make_features_shape,make_collocation_axis
import tensorflow as tf

In [11]:
make_features_shape(data)

{'num_samples': 10,
 'num_position_coords': 1681,
 'num_velocity_coords': 24,
 'num_phase_coords': 40344,
 'num_boundary_coords': 1968}

In [12]:
tf_data = np_to_tensor_dict(data)
tf.nest.map_structure(lambda x: x.shape, tf_data)

2022-11-23 21:45:37.221134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


{'sigma': TensorShape([10, 1681, 2]),
 'psi_label': TensorShape([10, 40344]),
 'boundary': TensorShape([10, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'scattering_kernel': TensorShape([10, 40344, 24]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}